In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import datetime as dt
from datetime import timedelta
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_samples
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score
import statsmodels.api as sm
from statsmodels.tsa.api import Holt,SimpleExpSmoothing,ExponentialSmoothing
from fbprophet import Prophet
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.stattools import adfuller
from pyramid.arima import auto_arima
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
std=StandardScaler()

/usr/local/lib/python3.5/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.

/usr/local/lib/python3.5/dist-packages/pyramid/__init__.py:68: UserWarning:


    The 'pyramid' package will be migrating to a new namespace beginning in 
    version 1.0.0: 'pmdarima'. This is due to a package name collision with the
    Pyramid web framework. For more information, see Issue #34:
    
        https:/

In [2]:
covid=pd.read_csv("covid_19_data.csv")
covid.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [3]:
# Check dataset size
print("Shape of the dataset: ",covid.shape)
print("Checking for null values:\n",covid.isnull().sum())
print("Checking Data-type of each column:\n",covid.dtypes)

Shape of the dataset:  (21220, 8)
Checking for null values:
 SNo                    0
ObservationDate        0
Province/State     10914
Country/Region         0
Last Update            0
Confirmed              0
Deaths                 0
Recovered              0
dtype: int64
Checking Data-type of each column:
 SNo                  int64
ObservationDate     object
Province/State      object
Country/Region      object
Last Update         object
Confirmed          float64
Deaths             float64
Recovered          float64
dtype: object


In [4]:
# Delete "SNo" column, because no use
covid.drop(["SNo"],1,inplace=True)
covid.head()

,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [5]:
# Convert "Observation Date" to Datetime format
covid["ObservationDate"]=pd.to_datetime(covid["ObservationDate"])
covid.head()

,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2020-01-22,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,2020-01-22,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,2020-01-22,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,2020-01-22,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [6]:
# Group the different types of cases as per date
datewise=covid.groupby(["ObservationDate"]).agg({"Confirmed":'sum',"Recovered":'sum',"Deaths":'sum'})
datewise["Days Since"]=datewise.index-datewise.index.min()

In [25]:
datewise

,Confirmed,Deaths,Recovered,Days Since,WeekOfYear
ObservationDate,,,,,
2020-01-22,555.0,17.0,28.0,0 days,4
2020-01-23,653.0,18.0,30.0,1 days,4
2020-01-24,941.0,26.0,36.0,2 days,4
2020-01-25,1438.0,42.0,39.0,3 days,4
2020-01-26,2118.0,56.0,52.0,4 days,4
2020-01-27,2927.0,82.0,61.0,5 days,5
2020-01-28,5578.0,131.0,107.0,6 days,5
2020-01-29,6165.0,133.0,126.0,7 days,5
2020-01-30,8235.0,171.0,143.0,8 days,5


In [7]:
print("Basic Information")
print("Totol number of countries with Disease Spread: ",len(covid["Country/Region"].unique()))
print("Total number of Confirmed Cases around the World: ",datewise["Confirmed"].iloc[-1])
print("Total number of Recovered Cases around the World: ",datewise["Recovered"].iloc[-1])
print("Total number of Deaths Cases around the World: ",datewise["Deaths"].iloc[-1])
print("Total number of Active Cases around the World: ",(datewise["Confirmed"].iloc[-1]-datewise["Recovered"].iloc[-1]-datewise["Deaths"].iloc[-1]))
print("Total number of Closed Cases around the World: ",datewise["Recovered"].iloc[-1]+datewise["Deaths"].iloc[-1])
print("Approximate number of Confirmed Cases per Day around the World: ",np.round(datewise["Confirmed"].iloc[-1]/datewise.shape[0]))
print("Approximate number of Recovered Cases per Day around the World: ",np.round(datewise["Recovered"].iloc[-1]/datewise.shape[0]))
print("Approximate number of Death Cases per Day around the World: ",np.round(datewise["Deaths"].iloc[-1]/datewise.shape[0]))
print("Approximate number of Confirmed Cases per hour around the World: ",np.round(datewise["Confirmed"].iloc[-1]/((datewise.shape[0])*24)))
print("Approximate number of Recovered Cases per hour around the World: ",np.round(datewise["Recovered"].iloc[-1]/((datewise.shape[0])*24)))
print("Approximate number of Death Cases per hour around the World: ",np.round(datewise["Deaths"].iloc[-1]/((datewise.shape[0])*24)))
print("Number of Confirmed Cases in last 24 hours: ",datewise["Confirmed"].iloc[-1]-datewise["Confirmed"].iloc[-2])
print("Number of Recovered Cases in last 24 hours: ",datewise["Recovered"].iloc[-1]-datewise["Recovered"].iloc[-2])
print("Number of Death Cases in last 24 hours: ",datewise["Deaths"].iloc[-1]-datewise["Deaths"].iloc[-2])

Basic Information
Totol number of countries with Disease Spread:  222
Total number of Confirmed Cases around the World:  3506729.0
Total number of Recovered Cases around the World:  1125236.0
Total number of Deaths Cases around the World:  247470.0
Total number of Active Cases around the World:  2134023.0
Total number of Closed Cases around the World:  1372706.0
Approximate number of Confirmed Cases per Day around the World:  34046.0
Approximate number of Recovered Cases per Day around the World:  10925.0
Approximate number of Death Cases per Day around the World:  2403.0
Approximate number of Confirmed Cases per hour around the World:  1419.0
Approximate number of Recovered Cases per hour around the World:  455.0
Approximate number of Death Cases per hour around the World:  100.0
Number of Confirmed Cases in last 24 hours:  79386.0
Number of Recovered Cases in last 24 hours:  32124.0
Number of Death Cases in last 24 hours:  3662.0


In [10]:
# Active Cases = Number of Confirmed Cases - Number of Recovered Cases - Number of Death Cases
fig=px.bar(x=datewise.index,y=datewise["Confirmed"]-datewise["Recovered"]-datewise["Deaths"])
fig.update_layout(title="Number of Active Cases",
                  xaxis_title="Tanggal",yaxis_title="Total Case",)
fig.show()

In [14]:
# Closed Cases = Number of Recovered Cases + Number of Death Cases
fig=px.bar(x=datewise.index,y=datewise["Recovered"]+datewise["Deaths"])
fig.update_layout(title="Number of Closed Cases",
                  xaxis_title="Tanggal",yaxis_title="Total Case")
fig.show()

In [15]:
datewise["WeekOfYear"]=datewise.index.weekofyear

In [16]:
datewise["WeekOfYear"]

ObservationDate
2020-01-22     4
2020-01-23     4
2020-01-24     4
2020-01-25     4
2020-01-26     4
2020-01-27     5
2020-01-28     5
2020-01-29     5
2020-01-30     5
2020-01-31     5
2020-02-01     5
2020-02-02     5
2020-02-03     6
2020-02-04     6
2020-02-05     6
2020-02-06     6
2020-02-07     6
2020-02-08     6
2020-02-09     6
2020-02-10     7
2020-02-11     7
2020-02-12     7
2020-02-13     7
2020-02-14     7
2020-02-15     7
2020-02-16     7
2020-02-17     8
2020-02-18     8
2020-02-19     8
2020-02-20     8
              ..
2020-04-04    14
2020-04-05    14
2020-04-06    15
2020-04-07    15
2020-04-08    15
2020-04-09    15
2020-04-10    15
2020-04-11    15
2020-04-12    15
2020-04-13    16
2020-04-14    16
2020-04-15    16
2020-04-16    16
2020-04-17    16
2020-04-18    16
2020-04-19    16
2020-04-20    17
2020-04-21    17
2020-04-22    17
2020-04-23    17
2020-04-24    17
2020-04-25    17
2020-04-26    17
2020-04-27    18
2020-04-28    18
2020-04-29    18
2020-04-30    1

In [17]:
for i in list(datewise["WeekOfYear"].unique()):
    print(i)

4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [20]:
datewise["WeekOfYear"]=datewise.index.weekofyear

week_num=[]
weekwise_confirmed=[]
weekwise_recovered=[]
weekwise_deaths=[]
w=1
for i in list(datewise["WeekOfYear"].unique()):
    weekwise_confirmed.append(datewise[datewise["WeekOfYear"]==i]["Confirmed"].iloc[-1])
    weekwise_recovered.append(datewise[datewise["WeekOfYear"]==i]["Recovered"].iloc[-1])
    weekwise_deaths.append(datewise[datewise["WeekOfYear"]==i]["Deaths"].iloc[-1])
    week_num.append(w)
    w=w+1

fig=go.Figure()
fig.add_trace(go.Scatter(x=week_num, y=weekwise_confirmed,
                    mode='lines+markers',
                    name='Weekly Growth of Confirmed Cases'))
fig.add_trace(go.Scatter(x=week_num, y=weekwise_recovered,
                    mode='lines+markers',
                    name='Weekly Growth of Recovered Cases'))
fig.add_trace(go.Scatter(x=week_num, y=weekwise_deaths,
                    mode='lines+markers',
                    name='Weekly Growth of Death Cases'))
fig.update_layout(title="Weekly Growth of different types of Cases in the world",
                 xaxis_title="Minggu ke",yaxis_title="Total case",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [21]:
# Scatter diagram
fig=go.Figure()
fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Confirmed"],
                    mode='lines+markers',
                    name='Confirmed Cases'))
fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Recovered"],
                    mode='lines+markers',
                    name='Recovered Cases'))
fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Deaths"],
                    mode='lines+markers',
                    name='Death Cases'))
fig.update_layout(title="Growth of cases",
                 xaxis_title="Tanggal",yaxis_title="Total case",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [22]:
print("Average increase in number of Confirmed Cases every day: ",np.round(datewise["Confirmed"].diff().fillna(0).mean()))
print("Average increase in number of Recovered Cases every day: ",np.round(datewise["Recovered"].diff().fillna(0).mean()))
print("Average increase in number of Deaths Cases every day: ",np.round(datewise["Deaths"].diff().fillna(0).mean()))

Average increase in number of Confirmed Cases every day:  34041.0
Average increase in number of Recovered Cases every day:  10924.0
Average increase in number of Deaths Cases every day:  2402.0


In [24]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Confirmed"].diff().fillna(0),mode='lines+markers',
                    name='Confirmed Cases'))
fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Recovered"].diff().fillna(0),mode='lines+markers',
                    name='Recovered Cases'))
fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Deaths"].diff().fillna(0),mode='lines+markers',
                    name='Death Cases'))
fig.update_layout(title="Daily increase in different types of Cases",
                 xaxis_title="Tanggal",yaxis_title="Total case",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [26]:
# Group by country
grouped_country=covid.groupby(["Country/Region","ObservationDate"]).agg({"Confirmed":'sum',"Recovered":'sum',"Deaths":'sum'})

In [27]:
grouped_country

Confirmed  Deaths  Recovered
Country/Region                 ObservationDate                              
 Azerbaijan                    2020-02-28             1.0     0.0        0.0
('St. Martin',)                2020-03-10             2.0     0.0        0.0
Afghanistan                    2020-02-24             1.0     0.0        0.0
                               2020-02-25             1.0     0.0        0.0
                               2020-02-26             1.0     0.0        0.0
                               2020-02-27             1.0     0.0        0.0
                               2020-02-28             1.0     0.0        0.0
                               2020-02-29             1.0     0.0        0.0
                               2020-03-01             1.0     0.0        0.0
                               2020-03-02             1.0     0.0        0.0
                               2020-03-03             1.0     0.0        0.0
                               2020-03-04             1.0     0.0        0.0
                               2020-03-05             1.0     0.0        0.0
                               2020-03-06             1.0     0.0        0.0
                               2020-03-07             1.0     0.0        0.0
                               2020-03-08             4.0     0.0        0.0
                               2020-03-09             4.0     0.0        0.0
                               2020-03-10             5.0     0.0        0.0
                               2020-03-11             7.0     0.0        0.0
                               2020-03-12             7.0     0.0        0.0
                               2020-03-13             7.0     0.0        0.0
                               2020-03-14            11.0     0.0        0.0
                               2020-03-15            16.0     0.0        0.0
                               2020-03-16            21.0     0.0        1.0
                               2020-03-17            22.0     0.0        1.0
                               2020-03-18            22.0     0.0        1.0
                               2020-03-19            22.0     0.0        1.0
                               2020-03-20            24.0     0.0        1.0
                               2020-03-21            24.0     0.0        1.0
                               2020-03-22            40.0     1.0        1.0
...                                                   ...     ...        ...
Zimbabwe                       2020-04-11            14.0     3.0        0.0
                               2020-04-12            14.0     3.0        0.0
                               2020-04-13            17.0     3.0        0.0
                               2020-04-14            17.0     3.0        0.0
                               2020-04-15            23.0     3.0        1.0
                               2020-04-16            23.0     3.0        1.0
                               2020-04-17            24.0     3.0        2.0
                               2020-04-18            25.0     3.0        2.0
                               2020-04-19            25.0     3.0        2.0
                               2020-04-20            25.0     3.0        2.0
                               2020-04-21            28.0     3.0        2.0
                               2020-04-22            28.0     4.0        2.0
                               2020-04-23            28.0     4.0        2.0
                               2020-04-24            29.0     4.0        2.0
                               2020-04-25            31.0     4.0        2.0
                               2020-04-26            31.0     4.0        2.0
                               2020-04-27            32.0     4.0        5.0
                               2020-04-28            32.0     4.0        5.0
                               2020-04-29            32.0     4.0        5.0
                               2020-04-30   